Student number:15203181
Student name:Jinghui Lu

Import mandatory packages.

In [1]:
import csv
import pandas as pd
import nltk
import sklearn
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from stop_words import get_stop_words
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import cross_validation
import numpy as np

**1.load the set of raw documents into notebook to create the dataset. **


Load data from various csv files.
I can load both **data and labels** directly from csv files cause I have added label to each row when I create csv files. 

In [2]:
try:
   
    df_b = pd.read_csv('business-raw.csv', header=None, sep=',', 
                 names=['label', 'description'])

    df_s = pd.read_csv('sport-raw.csv', header=None, sep=',', 
                 names=['label', 'description'])
except:
    print("Failed to get data!!!!")

Create three dataframes. One is a collection of sport news, one is a collection of business news,
And one is a collection of news from both topics.

In [3]:
#remove the duplicate data
df_sport=df_s.drop_duplicates()

#recreate the index
df_sport.index = range(len(df_sport))

#remove the duplicate data
df_business=df_b.drop_duplicates()

#recreate the index
df_business.index = range(len(df_business))

#create a whole corpus
corpus = df_business.append(df_sport)
corpus.index = range(len(corpus))

Check dataset.

In [4]:
#check dataset
print(corpus)

        label                                        description
0    business  Australian entrepreneur Craig Wright has publi...
1    business  Failed retailer BHS was 'crashed into a cliff'...
2    business  Yahoo boss Marissa Mayer will get $54.9m (£37....
3    business  Waiting staff should receive their tips in ful...
4    business  High Street chain Argos is immediately recalli...
5    business  Norway's sovereign wealth fund, the biggest of...
6    business  Australian Treasurer Scott Morrison's budget o...
7    business  Republican Party front-runner Donald Trump ste...
8    business  The head of a South Korean division of a UK co...
9    business  Leaked documents from EU-US trade talks reveal...
10   business  Vimeo will let video-makers set up their own s...
11   business  Japanese shares fall more than 3% at the start...
12   business  Puerto Rico's governor says the territory will...
13   business  Wales could have 50 vineyards in operation in ...
14   business  Australian

**2.create a document-term matrix, using appropriate text preprocessing and term weighting steps.**

Customerize a tokenizer function so that the corpus can be preprocessed, including **tokenize, downcase and Porterstem.**

In [5]:
#create a class for preprocessing corpus
class StemTokenizer(object):
     def __init__(self):
         self.ps = PorterStemmer()
     def __call__(self, doc):
         return [self.ps.stem(t) for t in nltk.word_tokenize(doc)] #tokenize,downcase and PorterStem each words

Customerize a stopwords sets including stopwords and punctuations.

In [6]:
#get stopwords from stop_words package, the list including punctuations and stopwords
stop_words = list(get_stop_words('english'))

#get all punctuations
punctuations = list(string.punctuation)

#merge stopwords and punctuations and some lost punctuaions or stopwords
merged = stop_words+punctuations+["''","'s"]

#check the set
print(merged)

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 't

Create document term matrix of **token counts** with all stopwords and punctuations removed for three datasets. All features are words which has been **tokenized, downcase and stem** and should appear in at least **two documents.**

In [7]:
#create a document term matrix vertorizer of token counts with all stopwords and punctuations removed
#All features are words which has been tokenized,downcase and stem 
#All words should appear in at least two documents
vectorizer = CountVectorizer(stop_words=merged, min_df = 2,tokenizer=StemTokenizer())


#create three document term matrix of different corpus
business_vec = vectorizer.fit_transform(df_business.description)
sport_vec = vectorizer.fit_transform(df_sport.description)
corpus_vec = vectorizer.fit_transform(corpus.description)

#see the terms of whole corpus
terms = vectorizer.get_feature_names()
print(terms)

['1', '1-1', '10', '100', '14', '2-2', '2016', '3-1', '49erfx', '``', 'access', 'accord', 'achiev', 'activ', 'admit', 'afternoon', 'ahead', 'aidan', 'alleg', 'allianz', 'alongsid', 'andrea', 'ankl', 'announc', 'annual', 'anoth', 'appl', 'appoint', 'around', 'atletico', 'australia', 'australian', 'author', 'back', 'bank', 'barclay', 'battl', 'bayern', 'bbc', 'beat', 'becaus', 'befor', 'behind', 'belfast', 'believ', 'betfr', 'bh', 'bid', 'big', 'biggest', 'bitcoin', 'boost', 'boss', 'brand', 'breath', 'breeder', 'brewster', 'brian', 'bridg', 'budget', 'busi', 'campaign', 'can', 'career', 'central', 'champion', 'championship', 'charg', 'chelsea', 'chester', 'china', 'chines', 'cian', 'citi', 'claim', 'clare', 'clash', 'classic', 'claudio', 'clearer', 'clinch', 'cloud', 'club', 'coach', 'colleagu', 'compani', 'complet', 'comprehens', 'comput', 'conduct', 'connacht', 'consum', 'continu', 'contract', 'control', 'counti', 'countri', 'craig', 'creator', 'cristiano', 'crown', 'crucibl', 'crunch

Check document term matrix.

In [8]:
#Check document term matrix of whole corpus
print(corpus_vec)

  (0, 31)	1
  (0, 128)	1
  (0, 97)	1
  (0, 419)	1
  (0, 179)	1
  (0, 50)	1
  (0, 98)	1
  (0, 327)	1
  (0, 253)	1
  (1, 419)	1
  (1, 138)	1
  (1, 312)	1
  (1, 46)	1
  (1, 402)	1
  (1, 60)	1
  (1, 350)	1
  (1, 38)	1
  (2, 52)	1
  (2, 412)	1
  (2, 167)	1
  (2, 277)	1
  (2, 230)	1
  (2, 207)	1
  (2, 390)	1
  (2, 148)	1
  :	:
  (161, 169)	1
  (161, 41)	1
  (161, 331)	1
  (161, 313)	1
  (161, 157)	1
  (161, 190)	1
  (161, 22)	1
  (161, 199)	1
  (161, 226)	1
  (162, 412)	1
  (162, 362)	1
  (162, 186)	1
  (162, 398)	1
  (162, 373)	1
  (162, 303)	1
  (162, 25)	1
  (162, 44)	1
  (162, 17)	1
  (162, 263)	1
  (162, 157)	1
  (162, 69)	1
  (163, 4)	1
  (163, 243)	1
  (163, 339)	1
  (163, 193)	1


List five most frequent words in **business** topic

In [9]:
freqs = business_vec.sum(axis=0)

sorted_term_indexes = freqs.argsort()
sorted_term_indexes = sorted_term_indexes[0, ::-1]

terms = vectorizer.get_feature_names()
for i in range(5):
    term_index = sorted_term_indexes[0,i]
    print( terms[term_index] )

divis
follow
finish
andrea
duo


List five most frequent words in **sport** topic

In [10]:
freqs = sport_vec.sum(axis=0)

sorted_term_indexes = freqs.argsort()
sorted_term_indexes = sorted_term_indexes[0, ::-1]

terms = vectorizer.get_feature_names()
for i in range(5):
    term_index = sorted_term_indexes[0,i]
    print( terms[term_index] )

deal
crucibl
entrepreneur
bank
clearer


List five most frequent words in the **whold corpus**

In [11]:
freqs = corpus_vec.sum(axis=0)

sorted_term_indexes = freqs.argsort()
sorted_term_indexes = sorted_term_indexes[0, ::-1]

terms = vectorizer.get_feature_names()
for i in range(5):
    term_index = sorted_term_indexes[0,i]
    print( terms[term_index] )

ha
hi
leagu
champion
will


Create document term matrix of **tfidf** with all stopwords and punctuations removed for three datasets.
All features are words which has been **tokenized, downcase and stem** and should appear in at least **two documents.**

In [12]:
#create a document term matrix vertorizer of tfidf with all stopwords and punctuations removed
#All features are words which has been tokenized,downcase and stem 
#All words should appear in at least two documents
vectorizer_tf = TfidfVectorizer(stop_words=merged, min_df = 2,tokenizer=StemTokenizer())

#create three document term matrix of different corpus
business_vec_tf = vectorizer_tf.fit_transform(df_business.description)
sport_vec_tf = vectorizer_tf.fit_transform(df_sport.description)
corpus_vec_tf = vectorizer_tf.fit_transform(corpus.description)

#see the terms of whole corpus
terms = vectorizer_tf.get_feature_names()
print(terms)

['1', '1-1', '10', '100', '14', '2-2', '2016', '3-1', '49erfx', '``', 'access', 'accord', 'achiev', 'activ', 'admit', 'afternoon', 'ahead', 'aidan', 'alleg', 'allianz', 'alongsid', 'andrea', 'ankl', 'announc', 'annual', 'anoth', 'appl', 'appoint', 'around', 'atletico', 'australia', 'australian', 'author', 'back', 'bank', 'barclay', 'battl', 'bayern', 'bbc', 'beat', 'becaus', 'befor', 'behind', 'belfast', 'believ', 'betfr', 'bh', 'bid', 'big', 'biggest', 'bitcoin', 'boost', 'boss', 'brand', 'breath', 'breeder', 'brewster', 'brian', 'bridg', 'budget', 'busi', 'campaign', 'can', 'career', 'central', 'champion', 'championship', 'charg', 'chelsea', 'chester', 'china', 'chines', 'cian', 'citi', 'claim', 'clare', 'clash', 'classic', 'claudio', 'clearer', 'clinch', 'cloud', 'club', 'coach', 'colleagu', 'compani', 'complet', 'comprehens', 'comput', 'conduct', 'connacht', 'consum', 'continu', 'contract', 'control', 'counti', 'countri', 'craig', 'creator', 'cristiano', 'crown', 'crucibl', 'crunch

Check document term matrix of tfidf.

In [13]:
#Check tfidf matrix of whole corpus
print(corpus_vec_tf)

  (0, 253)	0.370545893725
  (0, 327)	0.370545893725
  (0, 98)	0.349257234333
  (0, 50)	0.349257234333
  (0, 179)	0.207949854156
  (0, 419)	0.332744467239
  (0, 97)	0.332744467239
  (0, 128)	0.349257234333
  (0, 31)	0.307845304147
  (1, 38)	0.356203021018
  (1, 350)	0.396669455892
  (1, 60)	0.318970410967
  (1, 402)	0.301293490377
  (1, 46)	0.373879941608
  (1, 312)	0.373879941608
  (1, 138)	0.341759925251
  (1, 419)	0.356203021018
  (2, 148)	0.358827461894
  (2, 390)	0.380699465867
  (2, 207)	0.380699465867
  (2, 230)	0.380699465867
  (2, 277)	0.34186221759
  (2, 167)	0.380699465867
  (2, 412)	0.240364239677
  (2, 52)	0.34186221759
  :	:
  (161, 22)	0.330314634359
  (161, 190)	0.330314634359
  (161, 157)	0.296617420115
  (161, 313)	0.274421632517
  (161, 331)	0.250893149523
  (161, 41)	0.330314634359
  (161, 169)	0.311337347436
  (161, 127)	0.330314634359
  (161, 179)	0.185372125838
  (162, 69)	0.316622971706
  (162, 157)	0.301653141808
  (162, 263)	0.301653141808
  (162, 17)	0.3016531

List five most important words in **business** topic

In [19]:
freqs = business_vec_tf.sum(axis=0)

sorted_term_indexes = freqs.argsort()
sorted_term_indexes = sorted_term_indexes[0, ::-1]

terms = vectorizer_tf.get_feature_names()
for i in range(5):
    term_index = sorted_term_indexes[0,i]
    print( terms[term_index] )

divis
follow
cup
andrea
duo


List five most important words in **sport** topic

In [20]:
freqs = sport_vec_tf.sum(axis=0)

sorted_term_indexes = freqs.argsort()
sorted_term_indexes = sorted_term_indexes[0, ::-1]

terms = vectorizer_tf.get_feature_names()
for i in range(5):
    term_index = sorted_term_indexes[0,i]
    print( terms[term_index] )

deal
crucibl
bank
entrepreneur
european


List five most important words in the whold **corpus**

In [21]:
freqs = corpus_vec_tf.sum(axis=0)

sorted_term_indexes = freqs.argsort()
sorted_term_indexes = sorted_term_indexes[0, ::-1]

terms = vectorizer_tf.get_feature_names()
for i in range(5):
    term_index = sorted_term_indexes[0,i]
    print( terms[term_index] )

hi
ha
will
champion
leagu


**3.Build a classification model using a classifier**

Here we will create a **classifier** to predict the class of news.
First to split the corpus to train set and test set for training classifier model. Split rate is 0.2.

In [22]:
data_train, data_test, target_train, target_test=train_test_split(corpus.description, corpus.label, test_size=0.2)

Using Knn(n=3) to train the model

In [23]:
#create the train dataset
vectorizer = TfidfVectorizer()
train_X = vectorizer.fit_transform(data_train)

#create the model
model = KNeighborsClassifier(n_neighbors=3)
model.fit(train_X, target_train)

#create the test dataset using the same vocabulary
test_X = vectorizer.transform(data_test)

#predict the result
predicted = model.predict(test_X)

Check the result

In [24]:
print(predicted)

['business' 'sport' 'sport' 'business' 'business' 'sport' 'sport' 'sport'
 'business' 'sport' 'sport' 'business' 'business' 'sport' 'sport' 'sport'
 'sport' 'business' 'business' 'business' 'sport' 'business' 'business'
 'business' 'sport' 'business' 'business' 'sport' 'business' 'sport'
 'business' 'business' 'business']


**4.Test the predictions of the classification model using an appropriate evaluation strategy. Report and discuss results.**

Using matrix to compare the prediction result and the ground truth. 

We can find out the values on the diagonal from upper left to lower right is much bigger than the values on another diagonal which means the accuracy is really high.

In [25]:
cm = confusion_matrix(target_test, predicted,
labels=["business","sport"])
print(cm)

[[17  1]
 [ 1 14]]


We can also show the precision intuitively like below. In this case the precision is relatively high which means
the low false postive rate, namely the model performed well.

In [26]:
#show precision
precision_score(target_test, predicted, pos_label="business")

0.94444444444444442

We can also show the recall intuitively like below. In this case teh recall is relatively high which means low false negative rate, namely, the model performed well.

In [27]:
#show recall
recall_score(target_test, predicted, pos_label="business")

0.94444444444444442

We can also use f1-measure to evaluate the result, it is a trade-off between recall and precision.
The f1 measure values from the worst 0 to the best 1.
In this case the f1-measure is quite high which means the model in did well this case.

In [28]:
#show f1-measure score
f1_score(target_test, predicted, pos_label="business")

0.94444444444444442

Besides above evaluation methods, we can use **k-fold cross validation** to measure the model like below.
Here is a customerize function of k-fold validation.
The parameter are classifier,data,fold number, shuffle(True or False), label in sequence.
It will print the accuracy of each iteration and return a list of all accuracys.

In [29]:
def kf_cross_validation(clf,dataframe,fold,shuffle,label):
    #store the historic accuracy
    accuracys=[]
    #invoke KFold to seperate the dataset
    cv = cross_validation.KFold(len(dataframe), n_folds=fold, shuffle=shuffle, random_state=None)
    for train_index, test_index in cv:
        X_train, X_test = dataframe.description[train_index], dataframe.description[test_index]
        y_train, y_test = dataframe.label[train_index], dataframe.label[test_index]
        
        #create the train dataset
        vectorizer = TfidfVectorizer()
        train_X = vectorizer.fit_transform(X_train)

        #create the model
        model = clf
        model.fit(train_X, y_train)

        #create the test dataset using the same vocabulary
        test_X = vectorizer.transform(X_test)

        #predict the result
        predicted = model.predict(test_X)
        accuracy = precision_score(y_test, predicted, pos_label=label)
        print("Accuracy:",accuracy)
        
        #append the result to array
        accuracys.append(accuracy)
        
        
    return accuracys

Here we invoke the function and use knn(k=3) classifier,5 fold to evaluate the classifier.

In [30]:
scores = kf_cross_validation(KNeighborsClassifier(n_neighbors=3),corpus,5,True,"business")

Accuracy: 0.923076923077
Accuracy: 1.0
Accuracy: 0.9
Accuracy: 1.0
Accuracy: 0.923076923077


We can find out from the result that the overall accuracy is too high which means the model performed well.
But actually this result is a little tricky. Cause we can see some accuracy get to 1 and this is a obvious **overfitting result**.
I think the reason is there are too many dimension.**High-dimensionality** may cause Data has many irrelevant features
(dimensions) containing noise which leads to a poor model.
Maybe we should process the data using feature extraction or selection.

In [31]:
#see the average accuracy produced by cross validation
np.mean(scores)

0.94923076923076921